<a href="https://colab.research.google.com/github/abhinav9800/abhinav99/blob/main/sentiment_analysis_of_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import pandas as pd
from transformers import TFAutoModel, AutoTokenizer

In [67]:
df = pd.read_csv("/content/Tweets.csv")
df['label'] = df['label'].map({'positive': 1, 'negative': 0})

In [68]:
texts = df["tweet"].to_list()
labels = df["label"].to_list()
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [69]:
model = TFAutoModel.from_pretrained("bert-base-uncased",hidden_dropout_prob=0.2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [70]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [71]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [72]:
batch_size = 32

# Shuffle and batch the datasets
train_dataset = train_dataset.shuffle(len(train_texts)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [73]:
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=2)

In [74]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [75]:
classifier = BERTForClassification(model, num_classes=2)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [76]:
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=2)
history = classifier.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=6,
    callbacks=[early_stopping_callback]

)

Epoch 1/6
361/361 [==============================] - 217s 460ms/step - loss: 0.3374 - accuracy: 0.8522 - val_loss: 0.3089 - val_accuracy: 0.8688
Epoch 2/6
361/361 [==============================] - 164s 454ms/step - loss: 0.1862 - accuracy: 0.9296 - val_loss: 0.2883 - val_accuracy: 0.8823
Epoch 3/6
361/361 [==============================] - 165s 456ms/step - loss: 0.0681 - accuracy: 0.9797 - val_loss: 0.4133 - val_accuracy: 0.8719
Epoch 4/6
361/361 [==============================] - 165s 457ms/step - loss: 0.0283 - accuracy: 0.9930 - val_loss: 0.4386 - val_accuracy: 0.8781


In [77]:
classifier.evaluate(test_dataset)

91/91 [==============================] - 12s 137ms/step - loss: 0.4386 - accuracy: 0.8781


[0.43860337138175964, 0.8781163692474365]